In [ ]:
import string
import copy
import nltk
import os
import pandas as pd
import numpy as np
import pickle
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


Data Extraction 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
title = "Humor,Hist,Media,Food"

In [ ]:
paths = []
fileNameWithId={}
id=0
for (dirpath, dirnames, filenames) in os.walk(str('/content/drive/MyDrive/Humor,Hist,Media,Food')):
    for i in filenames:
        fileNameWithId[id]=i
        id=id+1
        paths.append(str(dirpath)+str("/")+i)

In [ ]:
fileNameWithId

In [ ]:
def remove_numbers(txt_data):
   numbers="0123456789"
   for i in range(len(numbers)):
       txt_data=np.char.replace(txt_data, numbers[i], ' ')
       txt_data = np.char.replace(txt_data, "  ", " ")
   return txt_data

In [ ]:
def punctuations_removal_from_data(txt_data):
    sym = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(sym)):
        txt_data = np.char.replace(txt_data, sym[i], ' ')
        txt_data = np.char.replace(txt_data, "  ", " ")
    txt_data = np.char.replace(txt_data, ',', ' ')
    txt_data = np.char.replace(txt_data, "'", "")#additionally removing apostrophe
    return txt_data

In [ ]:
def to_lower_case(txt_data):
    return np.char.lower(txt_data)

In [ ]:
def stop_words_removal_from_data(txt_data):
    stopWords = stopwords.words('english')
    words = word_tokenize(str(txt_data))
    clean_data = ""
    for w in words:
        if w not in stopWords:
            clean_data = clean_data + " " + w
    return np.char.strip(clean_data)

In [ ]:
def short_char_removal(txt_data):
    words = word_tokenize(str(txt_data))
    clean_data = ""
    for w in words:
        if len(w) > 1:
            clean_data = clean_data + " " + w
    return np.char.strip(clean_data)

In [ ]:
def lemmatize_text(txt_data):
    lemmatizer = WordNetLemmatizer()   
    tokens = word_tokenize(str(txt_data))
    clean_data = ""
    for w in tokens:
        clean_data = clean_data + " " +lemmatizer.lemmatize(w)
    return np.char.strip(clean_data)

Method for data Pre processing

In [ ]:
def process_txt_data(txt_data):
  txt_data=remove_numbers(txt_data)
  txt_data=punctuations_removal_from_data(txt_data)
  txt_data=to_lower_case(txt_data)
  txt_data=stop_words_removal_from_data(txt_data)
  txt_data=short_char_removal(txt_data)
  txt_data=lemmatize_text(txt_data)
  return txt_data

Intersection

In [ ]:
def IntersectionCount(query,document):
  a=set(query.tolist().split(" "))
  b=set(document.tolist().split(" "))
  return len(a&b)

Union

In [ ]:
def UnionCount(query,document):
  a=set(query.tolist().split(" "))
  b=set(document.tolist().split(" "))
  return len(a|b)

Method to calculate Jaccard Coeffecient

In [ ]:
def calculate_jaccard(query, document):
  intersection_count_jaccard = IntersectionCount(query, document)
  union_count_jaccard = UnionCount(query,document)
  jaccard_coeff = intersection_count_jaccard/union_count_jaccard
  return jaccard_coeff

In [ ]:
def find_Relevant_docs_using_jaccard_coeffecient(query):
  text_doc_processed=[]
  i=0
  processed_query = process_txt_data(query)
  jaccard_coeffecient = []
  for path in paths:
    file = open(path, 'r',encoding="cp850")
    text_data = file.read().strip()
    pre_processed_text=process_txt_data(text_data)
    jaccard_coeffecient.append(calculate_jaccard(processed_query,pre_processed_text))
    i+=1
    file.close()
  jaccard_coeffecient = np.array(jaccard_coeffecient)
  top_5_docs = jaccard_coeffecient.argsort()[-5:][::-1]
  for i in range(len(top_5_docs)):
    print(fileNameWithId[top_5_docs[i]])  

In [ ]:
query = input('Enter the Query   : ')
find_Relevant_docs_using_jaccard_coeffecient(query)

Enter the Query   : I am Sherlock
livnware.hum
mov_rail.txt
subb_lis.txt
crzycred.lst
insults1.txt


TFIDF

In [ ]:
doc_no = 0

tokens_in_docs = []

postings_list ={}
frequency_count = {}

for path in paths:
    file = open(path, 'r', encoding= 'cp850')
    text = " ".join(file.read().split()) 
    file.close()

    #preprocessing
    processed_doc = process_txt_data(text)  
    
    #tokenization
    generated_tokens = word_tokenize(str(processed_doc))
    tokens_in_docs.append(generated_tokens)
    position = 0

    #postings creation
    for token in generated_tokens:
        if token not in postings_list:
            postings_list[token]=[[[doc, {position}]]]
            frequency_count[token] = 1
            
        else:
            pos = postings_list[token]      
            d = [a[0] for a in pos]
            if doc_no in d:
                for a in pos:
                    if a[0] == doc:
                        a[1].add(position)
            else:
                pos.append([doc,{position}])
                frequency_count[token]+=1                           
        position += 1

    doc_no += 1


Raw Count Term Frequency

In [ ]:
freq_raw_count = []   #list to store raw count term frequency

#iterate in all docs 
for i in tokens_in_docs:
    raw_term_frequency = {}      #dictionary to store raw count term frequency
    for w in i:
        if raw_term_frequency.get(w) != None: 
            r = raw_term_frequency.get(w)             
            raw_term_frequency[w] = r+1                 
        else:
            raw_term_frequency[w] = 1 
    freq_raw_count.append(raw_term_frequency)     

Term Frequency

In [ ]:
def sum_values(list):
  sum=0
  for i in list:
    sum+=i
  return sum  

In [ ]:
freq_term_frequency = []    #list to store term frequency variant
for i in freq_raw_count:
    sum = sum_values(i.values())  
    term_freq = {}        #dictionary to store term frequency variant
    for w in i.keys():
        tf =(i.get(w)/sum) 
        term_freq[w] = tf
    freq_term_frequency.append(term_freq)

Log Normalization

In [ ]:
freq_log_normalization = []       #list to store log normalization term frequency
for i in freq_raw_count:
    log_normalization = {}         #dictionary to store log normalization term frequency
    for l in i.keys():
        log_normal = np.log(1+i.get(l))   
        log_normalization[l] = log_normal    
    freq_log_normalization.append(log_normalization)

Double Normalization

In [ ]:
freq_double_Normalisation = []  #list to store double normalization term frequency
for i in freq_raw_count:
    get_max = max(i.values())
    double_normalization = {}            #dictionary to store double normalization term frequency
    for l in i.keys():
        double_normalization[l] =  0.5 + (0.5*(i.get(l)/get_max))
    freq_double_Normalisation.append(double_normalization)

Binary 

In [ ]:
freq_binary = []   #list  to store binary term frequency
for i in freq_raw_count:
    binary = {}         #dictionary to store binary term frequency
    for l in i.keys():
        freq = i.get(l)
        if freq <= 0: 
            bool_freq = 0          
        else:
            bool_freq = 1  
        binary[l] = bool_freq
    freq_binary.append(binary)

No of docs in which a term 't' is present

In [ ]:

frequency_of_term_in_all_docs = {}    #dictionary to store frequnecy for each term in the doc
for i in tokens_in_docs:
   for l in list(set(i)):
        if frequency_of_term_in_all_docs.get(l) != None:                 
            frequency_of_term_in_all_docs[l] =  frequency_of_term_in_all_docs.get(l)+1
        else:
            frequency_of_term_in_all_docs[l] = 1  

IDF


In [ ]:
N = len(fileName_with_Id)
IDF = {}
for item in frequency_of_term_in_all_docs.keys():
    IDF[item] = np.log(N/frequency_of_term_in_all_docs[item]+1)

Method to calculate TF IDF values 

In [ ]:
def compute_tfidf(list_tf):
    list_tfidf = []  
    for i in list_tf:
        dict_tfidf = {}
        for l in i.keys():
            dict_tfidf[l] =  i[l] * IDF[l]
        list_tfidf.append(dict_tfidf)
    return list_tfidf

In [ ]:
#calculating tfidf for all 5 variants
binary_tf_idf = compute_tfidf(freq_binary)
raw_count_tf_idf = compute_tfidf(freq_raw_count)
term_frequency_tf_idf = compute_tfidf(freq_term_frequency)
log_normalisation_tf_idf = compute_tfidf(freq_log_normalization)
double_normalisation_tf_idf = compute_tfidf(freq_double_Normalisation)

List of All Terms

In [ ]:
all_terms = list(frequency_of_term_in_all_docs.keys())

TFIDF matrix using log

In [ ]:
log_matrix_tf_idf = []  #matrix to store tfidf values for log variant
for i in log_normalisation_tf_idf:
    log = []
    for l in all_terms:
        if l not in i :
            log.append(0.0)  
        else:
            log.append(i[l])  
            
    log_matrix_tf_idf += [log]  

In [ ]:
log_matrix_tf_idf[0]

TFIDF matrix using binary

In [ ]:
binary_matrix_tf_idf = []   #matrix to store tfidf values for binary variant
for i in binary_tf_idf:
    bin = []
    for l in all_terms:
        if l not in i :
            bin.append(0.0)   
        else:
            bin.append(i[l])
           
    binary_matrix_tf_idf += [bin]   

TFIDF matrix using Double Noralization

In [ ]:
#tf-idf score using double
double_matrix_tf_idf = []    #matrix to store tfidf values for double normalization
for i in double_normalisation_tf_idf:
    db = []
    for l in all_terms:
        if l not in i :
            db.append(0.0)   
        else:
           db.append(i[l]) 
           
    double_matrix_tf_idf += [db]   

TFIDF matrix using Term Frequency

In [ ]:
term_freq_matrix_tf_idf = []   #matrix to store tfidf values for term frequency
for i in term_frequency_tf_idf:
    term = []
    for l in all_terms:
        if l not in i :
            term.append(0.0)
             
        else:
            term.append(i[l]) 
              
    term_freq_matrix_tf_idf += [term]   

TFIDF matrix using Raw Count

In [ ]:
raw_matrix_tf_idf = []   #matrix to store tfidf values for raw count
for i in raw_count_tf_idf:
    raw = []
    for l in all_terms:
        if l not in i :
            raw.append(0.0)  
        else:
            raw.append(i[l])   
    raw_matrix_tf_idf += [raw]        

In [ ]:
import operator
def rank_doc_using_Tf_idf_values(variant_list,list_of_queries):

    vector_of_query = [0.0] * len(all_terms)
    result = [i for i, k in enumerate(all_terms) if k in set(list_of_queries)]
    print(result)

    for r in range(len(all_terms)):
       if r not in result:
        vector_of_query[r]=0.0    
       else:
        vector_of_query[r]=1.0
        print('Vector of Query : ',vector_of_query[r])           
    print('Vector of Query : ',vector_of_query)
        

    rank = {}
  
    for p in range(N):
        rank[p] = 0.0
    for word in result:
       
        count = -1
        for i in variant_list:
            count += 1
            rank[count] +=  i[word]
       
    rank = dict(sorted(rank.items(),key=operator.itemgetter(1),reverse=True))   
    print(rank)
    return rank,vector_of_query

In [ ]:
def top5(tfidfList,list_of_queries,k,var):

    tf_list ,vectorOfQuery= rank_doc_using_Tf_idf_values(tfidfList,list_of_queries)
    check = 0
    result = []
    cnt = -1    
    for i in tf_list.keys():
        cnt += 1
        if cnt == k:
            print('Top ',k,' Documents on the basis of ',var,' tf-idf values are',result)
            for i in result:
                print(fileName_with_Id.iloc[i][0])
            check = 1
            break
        result.append(i)
    if check == 0:
        print('Top ',k,' Documents on the basis of ',var,' tf-idf values are:',result)    
        for i in result:
            print(fileName_with_Id.iloc[i][0])
    return vectorOfQuery      

In [ ]:
def query_tokens(query):
    processed_query = process_txt_data(query)        #preprocessing of query
    query_token = word_tokenize(str(processed_query))   #token generation
    return query_token

In [ ]:
fileName_with_Id = pd.DataFrame(fileName_with_Id)

In [ ]:
query = input('Enter the String   : ')
#fetching all query vector
query_afterPreprocess =query_tokens(query)  # fetching query tokens
#fetching tfidf top 5 documents based on all 5 variants
query_vector_raw=top5(tfidf_raw_matrix,query_afterPreprocess,5,'Raw_Count')
query_vector_log=top5(tfidf_log_matrix,query_afterPreprocess,5,'Logarithmic')   
query_vector_term=top5(tfidf_term_matrix,query_afterPreprocess,5,'termfrequency')
query_vector_binary=top5(tfidf_binary_matrix,query_afterPreprocess,5,'Binary')
query_vector_double=top5(tfidf_double_matrix,query_afterPreprocess,5,'Double') 